# Sentiment Analysis

This notebook is the main one for this project. It will help navigate through the implementation scripts and notebooks, without going into too much detail.

## Preprocessing and Embeddings

The first part of this project was to find a big enough dataset. As mantioned in the notebook *"src/word2vec_training"*, different techniques were investigated. We finally decided to pick this dataset:

https://archive.org/details/archiveteam-twitter-stream-2017-11

From this base we filtered non english, truncated, retweeted or duplicate tweets. You can download the resulting dataset of 23M tweets we used here:

https://mega.nz/#!UI0ViKiZ!x6eBjFPmkKqDcV6Il-rpQj-DNcSJIOeL6Axk-vfuOyU

Here is a preprocessing pipeline example:

In [8]:
from functools import partial
from src.text_preprocessing import TweetTokenizer, NLTKStemmer, NLTKLemmatizer, CorpusWrapper

input_stream = [
    'This is a tweet',
    'This is another tweet',
]

factories = [
    TweetTokenizer,
    partial(CorpusWrapper, NLTKStemmer),
    partial(CorpusWrapper, NLTKLemmatizer),
    partial(BatchMaker, batch_size=100000),
]

batch_pipeline = Pipeline(input_stream, factories)

Using TensorFlow backend.


In [9]:
for batch in batch_pipeline:
    for tweet in batch:
        print(tweet)

['thi', 'be', 'a', 'tweet']
['thi', 'be', 'anoth', 'tweet']


We used Gensim to train embeddings on our dataset, using a previously defined pipeline. You can load our embeddings this way:

In [10]:
from gensim.models import Word2Vec

model = Word2Vec.load('./data/trained_embeddings_23M.model')

However, after testing, we noticed that the model for 3 classes was more performent with pre-trained embeddings on more than 330M tweets

In [ ]:
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.test.utils import get_tmpfile


tmp_file = get_tmpfile('datastories.300d.word2vec')
glove2word2vec('./data/embeddings/datastories.twitter.300d.txt', tmp_file)
w2v = KeyedVectors.load_word2vec_format(tmp_file)

## Emolex



## 3 class training

## Transfer learning

In [13]:
def reshape_model(model):
    model.pop()
    model.pop()
    for layer in model.layers:
        layer.trainable = False

    dense1 = Dense(150, activation='relu')
    dense3 = Dense(80, activation='relu')
    dense4 = Dense(30, activation='relu')
    dense2 = Dense(7, activation='softmax')

    model.add(dense1)
    model.add(Dropout(0.2))
    model.add(dense3)
    model.add(dense4)
    model.add(Flatten())
    model.add(dense2)

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
    return model

## TODO

- Emolex
  - improve this list
  - load the embedding matrix
- 3 class training
  - improve this list
  - show the Keras model
  - load the trained model
- Transfer learning
  - show the transformation function
  - model summary
  - show accuracy and pearson